In [1]:
from google.colab import drive
drive.mount('/content/dataset')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/dataset


In [2]:
from zipfile import ZipFile 
file_name="/content/dataset/My Drive/dataset/test.zip"
file_name2="/content/dataset/My Drive/dataset/train.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("extraction rèussi")

with ZipFile(file_name2,'r') as zip:
  zip.extractall()
  print("extraction rèussi")  



extraction rèussi
extraction rèussi


## Importing Basic Libraries and Installing Tools Required

In [3]:
!pip install memory_profiler
%tensorflow_version 1.x
import tensorflow as tf
%matplotlib inline
from memory_profiler import memory_usage
import os
import pandas as pd
from glob import glob
import numpy as np

  Created wheel for memory-profiler: filename=memory_profiler-0.57.0-cp36-none-any.whl size=28992 sha256=605d4b5ec594f6860bae05f895ab2fbae666046ef02986cc5c6edaa254a41ced
  Stored in directory: /root/.cache/pip/wheels/74/20/b5/20964ef97be73d2c3a695c9cad7bccd96d1e3e737a8163861f
Successfully built memory-profiler
TensorFlow 1.x selected.



### Installing libav-tools to get Librosa Working Properly

In [0]:
%%capture
!apt-get install libav-tools -y

### Importing Fast.ai for Deep Learning and Librosa for Creating Spectrogram

In [0]:
from fastai.vision import *
import librosa
import librosa.display
import pylab
import matplotlib
import gc

### Making Temporary Working Directories for Storing the Audio Conversions

## Defining the Create Spectrogram Function

In [0]:
def create_spectrogram(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = Path('/content/working/train/' + name + '.jpg')
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

In [0]:
def create_spectrogram_test(filename,name):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = Path('/content/working/test/'+ name + '.jpg')
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

### Splitting the Conversion in different cells and collecting the garbage to avoid Ram Overflow

In [0]:
Data_dir=np.array(glob("/content/Train/*"))

In [0]:
%load_ext memory_profiler

In [10]:
%memit 
i=0
for file in Data_dir[i:i+1500]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

peak memory: 547.49 MiB, increment: 0.05 MiB


In [11]:
gc.collect()

21918

In [12]:
%memit 
i=1500
for file in Data_dir[i:i+1500]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

peak memory: 673.02 MiB, increment: 0.00 MiB


In [13]:
gc.collect()

20232

In [14]:
%memit 
i=3000
for file in Data_dir[i:i+1500]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

peak memory: 673.97 MiB, increment: 0.00 MiB


In [15]:
gc.collect()

20232

In [16]:
%%memit 
i=4500
for file in Data_dir[i:]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram(filename,name)

peak memory: 687.21 MiB, increment: 4.51 MiB


In [17]:
gc.collect()

8430

## Creating Data Bunch for Training and Creating a Resnet34 Model

In [0]:
path = Path('/content/working/')
np.random.seed(42)
data = ImageDataBunch.from_csv(path,csv_labels='/content/train.csv', folder="train", valid_pct=0.2, suffix='.jpg',
        ds_tfms=get_transforms(), size=224, num_workers=0).normalize(imagenet_stats)

In [0]:
data.classes

['air_conditioner',
 'car_horn',
 'children_playing',
 'dog_bark',
 'drilling',
 'engine_idling',
 'gun_shot',
 'jackhammer',
 'siren',
 'street_music']

In [19]:
learn = create_cnn(data, models.resnet34, metrics=accuracy)

/usr/local/lib/python3.6/dist-packages/fastai/vision/learner.py:106: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")
Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth


## Training The Model

In [20]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy,time
0,1.877404,0.866809,0.702852,01:01
1,1.225588,0.663163,0.771849,01:00
2,0.886054,0.562463,0.816007,00:59
3,0.756956,0.529082,0.819687,00:59


In [0]:
learn.save('stage-1')

In [22]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [23]:
learn.fit_one_cycle(4, max_lr=slice(1e-4,1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.590528,0.494499,0.852806,01:06
1,0.472383,0.373440,0.885925,01:06
2,0.278796,0.198899,0.934683,01:07
3,0.165708,0.142852,0.955842,01:07


In [0]:
learn.save('stage-2')

In [25]:
learn.lr_find()
learn.recorder.plot()

epoch,train_loss,valid_loss,accuracy,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [26]:
learn.fit_one_cycle(1, max_lr=slice(1e-6,2e-6))

epoch,train_loss,valid_loss,accuracy,time
0,0.106049,0.131843,0.962282,01:07


In [0]:
learn.save('stage-3')

In [0]:
Test_dir=np.array(glob("/content/Test/*"))

In [29]:
%%memit 
i=0
for file in Test_dir[i:i+1500]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram_test(filename,name)

/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel 

peak memory: 2931.05 MiB, increment: 46.53 MiB


In [30]:
gc.collect()

47208

In [31]:
%%memit 
i=1500
for file in Test_dir[i:]:
    filename,name = file,file.split('/')[-1].split('.')[0]
    create_spectrogram_test(filename,name)

/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
/usr/local/lib/python3.6/dist-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel 

peak memory: 2932.61 MiB, increment: 4.55 MiB


In [32]:
gc.collect()

10025

In [0]:
learn.load('stage-3')
test_csv = pd.read_csv('/content/test.csv')

## Making predictions and writing it to CSV

In [0]:
with open('output.csv',"w") as file:
    file.write("ID,Prediction\n")
    for test in test_csv.ID:
        img = open_image('/content/working/test/'+str(test)+'.jpg')
        prediction = str(learn.predict(img)[0]).split()[0]
        file.write(str(test)+','+prediction)
        file.write('\n')

In [35]:
output = pd.read_csv('output.csv')
output.head()

,ID,Prediction
0,5,jackhammer
1,7,dog_bark
2,8,drilling
3,9,dog_bark
4,13,street_music
